In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("output.csv")

In [4]:
predictors = ["surface_code","pl1_hand_code","pl2_hand_code","round_code","court_code","series_code","best_of","pl1_rank","pl2_rank","pl1_pts","pl2_pts","pl1_year_pro","pl2_year_pro","pl1_weight","pl2_weight","pl1_height","pl2_height",'pl1_avg_bookmaker_odds','pl2_avg_bookmaker_odds','pl1_total_games','pl1_win_rate','pl2_total_games','pl2_win_rate','pl1_swrate','pl2_swrate','rank_diff','pts_diff','wrate_diff','swrate_diff']

train = df[df["date"] < "2019-01-01"]
test = df[df["date"] >= "2019-01-01"]

X_train, y_train = train[predictors], train['result']
X_test, y_test = test[predictors], test['result']

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
# tvorba Random Forest klasifikátoru
rf_classifier = RandomForestClassifier(random_state=42)

# trénink modelu
rf_classifier.fit(X_train, y_train)

# tvorba predikcí na testovacích vzorku
y_pred = rf_classifier.predict(X_test)

# zhodnocení přenosti modelu
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# zobrazení významnosti jednotlivých prediktorů
feature_importance_df = pd.DataFrame({
    'Feature': predictors,
    'Importance': rf_classifier.feature_importances_
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance_df)

Accuracy: 0.6704

Feature Importance:
                   Feature  Importance
18  pl2_avg_bookmaker_odds    0.113716
17  pl1_avg_bookmaker_odds    0.101503
24              pl2_swrate    0.055045
28             swrate_diff    0.053020
23              pl1_swrate    0.050358
27              wrate_diff    0.044762
9                  pl1_pts    0.043546
26                pts_diff    0.042984
20            pl1_win_rate    0.042919
10                 pl2_pts    0.041634
22            pl2_win_rate    0.039784
7                 pl1_rank    0.039401
8                 pl2_rank    0.039083
25               rank_diff    0.035705
21         pl2_total_games    0.033417
19         pl1_total_games    0.033032
14              pl2_weight    0.024367
13              pl1_weight    0.024319
12            pl2_year_pro    0.024075
11            pl1_year_pro    0.024062
16              pl2_height    0.020219
15              pl1_height    0.019769
3               round_code    0.014428
5              series_code

In [18]:
predicted_values = rf_classifier.predict(X_test)

print(predicted_values)

[0 0 0 ... 1 0 0]


In [10]:
test_df['prediction'] = predicted_values

C:\Users\pariz\AppData\Local\Temp\ipykernel_31588\1293353320.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['prediction'] = predicted_values


In [11]:
test_df.to_excel('rf_output.xlsx', index=False)